# Case study, How does a bike-share navigate speedy success?
<p style="align, center"><img src="./misc/bike.JPG"></p>

## Business Task
Design marketing strategies aimed at converting casual riders into annual members

## Analytic questions:
* How do annual members and casual riders use Cyclistic bikes differently?
* Why would casual riders buy Cyclistic annual memberships? 
* How can Cyclistic use digital media to influence casual riders to become members?

### Source
* 202401-divvy-tripdata.csv
* 202402-divvy-tripdata.csv
* 202403-divvy-tripdata.csv
* 202404-divvy-tripdata.csv
* 202405-divvy-tripdata.csv
* 202406-divvy-tripdata.csv
* 202407-divvy-tripdata.csv
* 202408-divvy-tripdata.csv
* 202409-divvy-tripdata.csv
* 202410-divvy-tripdata.csv
* 202411-divvy-tripdata.csv
* 202412-divvy-tripdata.csv

<p style="font-style: italic;">The data has been made available by Motivate International Inc. under this <a href="https://divvybikes.com/data-license-agreement">license</a>.</p>

### Libraries import

In [68]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('ggplot')

## Preparation
### Load raw data

In [69]:
csv_files = glob.glob('raw data/*.csv')
dfs = []

for file in csv_files:
    data = pd.read_csv(file, encoding="utf-8",) 
    dfs.append(data)

df = pd.concat(dfs)
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member


### Loading revelant columns

In [70]:
revelant_cols_df = df[['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'end_station_name', 'member_casual']].copy()
revelant_cols_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,Kingsbury St & Kinzie St,member
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,Kingsbury St & Kinzie St,member
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,Kingsbury St & Kinzie St,member
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,Larrabee St & Webster Ave,member
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,Kingsbury St & Kinzie St,member


### Efficient datatypes conversion 

In [71]:
schema = {
    'ride_id': 'string[pyarrow]',
    'rideable_type': 'category',
    'started_at': 'datetime64[ns]',
    'ended_at': 'datetime64[ns]',
    'start_station_name': 'category',
    'end_station_name': 'category',
    'member_casual': 'category'
}

cast_df = revelant_cols_df.astype(schema).copy()
cast_df.dtypes

ride_id               string[pyarrow]
rideable_type                category
started_at             datetime64[ns]
ended_at               datetime64[ns]
start_station_name           category
end_station_name             category
member_casual                category
dtype: object

### Memory usage comparison

In [72]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 5860568 entries, 0 to 178371
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 3.3 GB


In [73]:
cast_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 5860568 entries, 0 to 178371
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             string        
 1   rideable_type       category      
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  category      
 5   end_station_name    category      
 6   member_casual       category      
dtypes: category(4), datetime64[ns](2), string(1)
memory usage: 302.2 MB


## Process

### Preliminary data analysis

In [74]:
cast_df.shape

(5860568, 7)

In [75]:
cast_df.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,Kingsbury St & Kinzie St,member
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,Kingsbury St & Kinzie St,member
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,Kingsbury St & Kinzie St,member
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,Larrabee St & Webster Ave,member
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,Kingsbury St & Kinzie St,member
5,C96080812CD285C5,classic_bike,2024-01-07 11:21:24,2024-01-07 11:30:03,Wells St & Elm St,Kingsbury St & Kinzie St,member
6,0EA7CB313D4F456A,classic_bike,2024-01-05 14:44:12,2024-01-05 14:53:06,Wells St & Elm St,Kingsbury St & Kinzie St,member
7,EE11F3A3B39CFBD8,electric_bike,2024-01-04 18:19:53,2024-01-04 18:28:04,Wells St & Elm St,Kingsbury St & Kinzie St,member
8,63E83DE8E3279F15,classic_bike,2024-01-01 14:46:53,2024-01-01 14:57:02,Wells St & Elm St,Kingsbury St & Kinzie St,member
9,8005682869122D93,electric_bike,2024-01-03 19:31:08,2024-01-03 19:40:05,Clark St & Ida B Wells Dr,Kingsbury St & Kinzie St,member


In [76]:
cast_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'end_station_name', 'member_casual'],
      dtype='object')

In [77]:
cast_df.describe()

,started_at,ended_at
count,5860568,5860568
mean,2024-07-17 07:55:47.617262848,2024-07-17 08:13:06.552330496
min,2024-01-01 00:00:39,2024-01-01 00:04:20
25%,2024-05-20 19:47:53,2024-05-20 20:07:54.750000128
50%,2024-07-22 20:36:16.283500032,2024-07-22 20:53:59.158500096
75%,2024-09-17 20:14:22.566249984,2024-09-17 20:27:46.025999872
max,2024-12-31 23:56:49.854000,2024-12-31 23:59:55.705000


### Check for N/A values and duplicates

In [78]:
cast_df.isna().sum()

ride_id                     0
rideable_type               0
started_at                  0
ended_at                    0
start_station_name    1073951
end_station_name      1104653
member_casual               0
dtype: int64

Assuming that rides without registered start or end station are registered by errors thus we won't take them into analysis

In [79]:
clean_df = cast_df.dropna().copy()

Checking for rides with the same ride_id

In [87]:
clean_df.loc[clean_df.duplicated(subset=['ride_id'], keep=False)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
943,2C772EDDDBDEFDA3,electric_bike,2024-05-31 23:42:42.000,2024-06-01 00:25:08.000,Dearborn St & Van Buren St,DuSable Lake Shore Dr & Monroe St,casual
17181,7BC67FD33887B3CB,classic_bike,2024-05-31 22:54:20.000,2024-06-01 00:42:41.000,Burnham Harbor,St. Louis Ave & Fullerton Ave,casual
20441,43CD52984AD22D99,electric_bike,2024-05-31 23:45:51.000,2024-06-01 00:11:46.000,Wentworth Ave & Cermak Rd*,Loomis St & Lexington St,member
22451,ABBD88BEBC1431FF,electric_bike,2024-05-31 23:20:35.000,2024-06-01 00:33:41.000,McClurg Ct & Erie St,McClurg Ct & Erie St,casual
25437,CA12CCDD359DA80C,classic_bike,2024-05-31 23:54:35.000,2024-06-01 00:01:25.000,Halsted St & Archer Ave,Morgan St & 31st St,casual
...,...,...,...,...,...,...,...
627078,2F6D74102A4FFFD6,classic_bike,2024-05-31 23:45:59.789,2024-06-01 00:11:51.768,Morgan St & Lake St*,LaSalle Dr & Huron St,member
627085,6ED3DDF49B9EE461,classic_bike,2024-05-31 23:48:38.271,2024-06-01 00:13:31.085,Morgan St & Lake St*,LaSalle Dr & Huron St,casual
627733,43637BA11F2DAA42,classic_bike,2024-05-31 20:27:14.662,2024-06-01 13:28:51.261,Dearborn St & Erie St,Broadway & Cornelia Ave,casual
632558,07DBFDA3C91006AE,classic_bike,2024-05-31 22:35:31.362,2024-06-01 09:05:13.160,Western Ave & Walton St,Western Ave & Walton St,casual


In [89]:
clean_df = clean_df.drop_duplicates(subset=['ride_id']).copy()

Checking for stations that have same start time, end time, start_station and end_station_name

In [90]:
clean_df.loc[clean_df.duplicated(subset=['started_at', 'ended_at', 'start_station_name', 'end_station_name'], keep=False)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
102692,D3CE8F069C526B23,classic_bike,2024-01-30 20:27:57,2024-01-30 20:50:51,Canal St & Adams St,State St & Randolph St,member
102754,BD06A0FDAC1D1183,classic_bike,2024-01-30 20:27:57,2024-01-30 20:50:51,Canal St & Adams St,State St & Randolph St,member
71207,A47F1F8E79A156BC,classic_bike,2024-02-27 13:40:53,2024-02-27 13:57:21,McClurg Ct & Erie St,Halsted St & Clybourn Ave,casual
165886,36EE4451474A3DC0,classic_bike,2024-02-27 13:40:53,2024-02-27 13:57:21,McClurg Ct & Erie St,Halsted St & Clybourn Ave,casual


In [91]:
clean_df = clean_df.drop_duplicates(subset=['started_at', 'ended_at', 'start_station_name', 'end_station_name']).copy()

Checking for stations that have same start time, end time, start_station or end_station_name
* Whenever those records need to be deleted, consultation with someone from the company is required to determine if the given station can hold more than two bikes.

In [92]:
clean_df.loc[clean_df.duplicated(subset=['started_at', 'ended_at', 'start_station_name'], keep=False)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual


In [93]:
clean_df.loc[clean_df.duplicated(subset=['started_at', 'ended_at', 'end_station_name'], keep=False)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
102616,48AB680D63B319F5,classic_bike,2024-03-04 17:36:30,2024-03-04 17:46:12,Mies van der Rohe Way & Chestnut St,New St & Illinois St,member
181898,FAA4EB520E8918AF,classic_bike,2024-03-04 17:36:30,2024-03-04 17:46:12,Wells St & Huron St,New St & Illinois St,member
